# TriScale Demo

This notebook contains a short demonstration of _TriScale_ API, how the different functions are meant to be used, and the visualizations they produce.

- [List of Imports](#List-of-Imports)
- [Experiment Sizing](#Experiment-Sizing)
- [Runs and Metrics](#Runs-and-Metrics)
- [Series and KPIs](#Series-and-KPIs)
- [Sequels and Variability Scores](#Sequels-and-Variability-Scores)
- [Network Profiling](#Network-Profiling)

## List of Imports

In [1]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

import triscale

%load_ext autoreload
%autoreload 2

## Experiment Sizing

During the design phase of an experiment, one important question to answer is "how many time should the experiments be performed?" This question directly relates to the definition of _TriScale_ KPIs and variability scores. 

_TriScale_ implements a statistical method that allows to estimate, based on a data sample, any percentile of the underlying distribution with any level of confidence. Importantly, the estimation does not rely on any assumption on the nature of the underlying distribution (eg normal, or Poisson). The estimate is valid as long as the sample is independent and identically distributed (_i.i.d._).

Intuitively, it is "easier" to estimate the median (50th percentile) than the 99th percentile; the more extreme the percentile, the more samples are required to provide an estimate for a given level of confidence. More precisely, the minimal number of sample $N$ required to estimate a percentile $0<p<1$ with confidence $0<C<1$ is given by:

$$N \;\geq\; \frac{log(1-C)}{log(1-p)}$$

_TriScale_ `experiment_sizing()` function implements this computation and retuens the minimal number of samples $N$, as illustrated below.

In [9]:
# Select the percentile we want to estimate 
percentile = 10

# Select the desired level of confidence for the estimation
confidence = 99 # in %

# Compute the minimal number of samples N required
triscale.experiment_sizing(
    percentile, 
    confidence,
    verbose=True); 

A one-sided bound of the 	10-th percentile
with a confidence level of	99 % 
requires a minimum of 		44 samples



Let us consider the samples $x$ are ordered such that $x_1 \leq x_2 \ldots \leq x_N$. 
The previous result indicates that for $N = 44$ samples and above,  $x_1$ is a lower bound for the 10th percentile with probibility larger than 99%.

To get a better feeling of how this minimal number of samples evolves this increasing confidence and more extreme percentiles, let us compute a range of minimal number of samples and display the results in a table (where the columns are the percentiles to estimate).

In [10]:
percentiles = [0.1, 1, 5, 10, 25, 50, 75, 90, 95, 99, 99.9]
confidences = [75, 90, 95, 99, 99.9, 99.99]
min_number_samples = []

for c in confidences:
    tmp = []
    for p in percentiles:
        N = triscale.experiment_sizing(p,c)
        tmp.append(N[0])
    min_number_samples.append(tmp)
    
df = pd.DataFrame(columns=percentiles, data=min_number_samples)
df['Confidence level'] = confidences
df.set_index('Confidence level', inplace=True)

display(df)

,0.1,1.0,5.0,10.0,25.0,50.0,75.0,90.0,95.0,99.0,99.9
Confidence level,,,,,,,,,,,
75.00,1386,138,28,14,5,2,5,14,28,138,1386
90.00,2302,230,45,22,9,4,9,22,45,230,2302
95.00,2995,299,59,29,11,5,11,29,59,299,2995
99.00,4603,459,90,44,17,7,17,44,90,459,4603
99.90,6905,688,135,66,25,10,25,66,135,688,6905
99.99,9206,917,180,88,33,14,33,88,180,917,9206


Similarly, one can compute the minimal number $N$ such that any sample $x_m$ is an estimate (instead of $x_1$). This can be obtained from the `experiment_sizing()` function using the option `robustness` argument.

In [13]:
triscale.experiment_sizing(
    percentile, 
    confidence,
    robustness=3,
    verbose=True); 

A one-sided bound of the 	10-th percentile
with a confidence level of	99 % 
requires a minimum of 		97 samples
with the worst 			3 run(s) excluded



The previous result indicates that for $N = 97$ samples and above,  $x_4$ is a lower bound for the 10th percentile with probibility larger than 99%.

## Runs and Metrics

Metrics in _TriScale_ evaluate a performance dimension across a run. The computation of metrics is implmented in the `analysis_metric()` functions, which takes two compulsory arguments:
- the raw data,
- the metric definition,

The raw data can be passed as a file name (ie, a string) or as a Pandas dataframe. 
- If a string is passed, the function tries to read the file name as a csv file (comma separated) with `x` data in the first column and `y` data in the second column. 
- If a pandas DataFrame is passed, `data` must contain columns named `x` and `y`.

The metric definition is provided as a dictionary, with only the `measure` key being compulsory. This defines "what is the computation to be performed" on the data. The measure can be any percentile ($0<P<100$) or `mean`, `minimum`, `maximum`.

In [27]:
# Input data file
data = 'ExampleData/bbr_datalink_delay_run9_flow1.csv'

# Definition of a TriScale metric
metric = {  
    'measure': 50,   # Integer: interpreted as a percentile
    'unit'   : 'ms', # For display only
         }

has_converged, metric_measure, plot = triscale.analysis_metric( 
    data,
    metric)

print('Run metric: %0.2f %s' % (metric_measure, metric['unit']))

Run metric: 66.10 ms


This computation per se is not very interesting. The main value of the `analysis_metric()` function is when the metric attempts to estimate the long-term performance; that is, the value one would obtain shall the run last longer/more data points be collected. 

This is performed by passing the optional `convergence` parameter; the function then integrates the _TriScale_ convergence test, described in details in [the paper](https://doi.org/10.5281/zenodo.3464273). Passing `plot=True` triggers the plotting of the raw data, the metric data, and the convergence test data.

In [34]:
# Input data file
data = 'ExampleData/bbr_datalink_delay_run9_flow1.csv'

# Definition of a TriScale metric
metric = {  
    'measure': 50,   # Integer: interpreted as a percentile
    'unit'   : 'ms', # For display only
         }

# Parameters for the convergence test
convergence = {'expected': True}

has_converged, metric_measure, plot = triscale.analysis_metric( 
    data,
    metric,
    convergence=convergence,
    plot=True,
)
if has_converged:
    print('The metric data has converged.')
    print('Run metric: %0.2f %s' % (metric_measure, metric['unit']))
else:
    print('The metric data has **not** converged.')

The metric data has converged.
Run metric: 66.78 ms


Let us zoom-in on the Y-axis to see the data from the convergence test better...

In [35]:
plot.update_layout(yaxis_range=[64,68])
plot.show()

As detailed in [the paper](https://doi.org/10.5281/zenodo.3464273), the `Metric` data are computed over a sliding window of the raw data points (`Data`). _TriScale_ convergence test constists in performing a linear regression (`Slope`). TriScale defines that a run _has converged_ when the slope of the linear regression is "sufficiently close" to 0. 
_TriScale_ formalizes "sufficiently close" as follows: the confidence interval for the slope must fall within the tolerance values. The confidence interval on the slope is computed using [bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)). _TriScale_ convergence tests uses default values of 95% confidence level and 5% tolerence. This defaults can be overwritten by the user as shown below.

In [36]:
# Input data file
data = 'ExampleData/bbr_datalink_delay_run9_flow1.csv'

# Definition of a TriScale metric
metric = {  
    'measure': 50,   # Integer: interpreted as a percentile
    'unit'   : 'ms', # For display only
         }

# Parameters for the convergence test
convergence = {
    'expected'  : True,
    'confidence': 75,
    'tolerance' : 1
}

# Customized plot layout
layout = dict(yaxis=dict(range=[64,68]))

has_converged, metric_measure, plot = triscale.analysis_metric( 
    data,
    metric,
    convergence=convergence,
    plot=True,
    custom_layout=layout
)
if has_converged:
    print('The metric data has converged.')
    print('Run metric: %0.2f %s' % (metric_measure, metric['unit']))
else:
    print('The metric data has **not** converged.')

The metric data has converged.
Run metric: 66.78 ms


One can observe that the tolerance interval got slimer (1% instead of 5%). Similarly, the confidence interval on the slope got slimmer too, as we decrease the confidence level from 95% to 75% (that is, the shadded area is expected to contain the true slope value with 75% probability).  

## Series and KPIs
Work in progress.

## Sequels and Variability Scores

Work in progress.

## Network Profiling
Work in progress.

In [37]:
# Recompute
# data_file = Path('UseCase_Glossy/Data_FlockLab/2019-08_FlockLab_sky.csv')
# df = flocklab.parse_data_file(str(data_file), active_link_threshold=50)

# Load
df = pd.read_csv('ExampleData/network_profiling.csv')

link_quality_bounds = [0,100]
link_quality_name = 'PRR [%]'

# Produce the plot
fig_theil, fig_autocorr = triscale.network_profiling(
                            df, 
                            link_quality_bounds, 
                            link_quality_name,
                            )
fig_autocorr.show()
fig_theil.show()